In [1]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
import numpy as np
from operator import itemgetter

import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy.sparse as sps

In [2]:
from Data_Handler.DataReader import DataReader 
dataReader=DataReader()
URM_all=dataReader.load_augmented_binary_urm()
powerful_urm=dataReader.load_powerful_binary_urm()

#URM_train, URM_validation = split_train_in_two_percentage_global_sample(urm, train_percentage = 0.9)
#train_powerful, val_powerful = split_train_in_two_percentage_global_sample(powerful_urm, train_percentage = 0.9)

In [3]:
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 318 ( 0.8%) Users that have less than 1 test interactions


In [9]:
MAP=[]

for topK in [10, 50, 100, 200, 500]:
    for shrink in [0, 10, 50, 100, 200, 500]:
        recommender = ItemKNNCFRecommender(URM_train)
        recommender.fit(topK=topK, shrink=shrink)

        result_dict, _ = evaluator_validation.evaluateRecommender(recommender)

        MAP.append(('topK={}'.format(topK),'shrink={}'.format(shrink),result_dict[10]["MAP"]))

Unable to load Cython Compute_Similarity, reverting to Python
Similarity column 24507 (100.0%), 1425.23 column/sec. Elapsed time 17.20 sec
EvaluatorHoldout: Processed 41311 (100.0%) in 14.99 sec. Users per second: 2757


KeyError: 10

In [ ]:
max_tuple = max(MAP, key=itemgetter(2))
print('The best combination for the ICF recommender is ({},{}), with a MAP = {}'.format(max_tuple[0],max_tuple[1], max_tuple[2]))

In [6]:
MAP=[]

for num_factors in np.logspace(1,15,10, base=2).astype(int):
    recommender = PureSVDRecommender(URM_train)
    recommender.fit(num_factors = num_factors)

    result_dict, _ = evaluator_validation.evaluateRecommender(recommender)

    MAP.append(('number of factors={}'.format(num_factors),result_dict[10]["MAP"]))

PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing SVD decomposition... done in 0.27 sec
EvaluatorHoldout: Processed 41311 (100.0%) in 22.34 sec. Users per second: 1849


KeyError: 10